In [10]:
import numpy as np
import networkx as nx
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
from dataclasses import dataclass
import copy          
import math
import itertools
from collections import defaultdict
import pandas as pd
import numpy as np
import re
import csv
import os
import sys, io, os, textwrap
from datetime import datetime



@dataclass
class SewerLink:
    """Represents a sewer link with its properties"""
    up_node: int
    down_node: int
    length: float
    diameter: float = 0.2  # meters
    slope: float = 0.001  # 1 in 1000
    flow: float = 0.0  # m3/s
    
@dataclass
class Node:
    """Represents a manhole node"""
    id: int
    wastewater_contribution: float  # l/s
    ground_level: float = 0.0
    invert_level: float = 0.0

class SpanningTreeGenerator:
    """Generates predetermined number of spanning trees in order of increasing length"""
    
    def __init__(self, base_graph: nx.Graph, outlet_node: int):
        self.base_graph = base_graph
        self.outlet_node = outlet_node
        self.spanning_trees = []
        
    def generate_spanning_trees(self, n_trees: int) -> List[nx.Graph]:
        """Generate n spanning trees in order of increasing total length"""
        all_trees = []
        seen_trees = set()
        
        # Generate MST as first tree
        mst = nx.minimum_spanning_tree(self.base_graph, weight='length')
        tree_signature = self._get_tree_signature(mst)
        all_trees.append((self._calculate_total_length(mst), mst))
        seen_trees.add(tree_signature)
        
        print(f"Generated MST with length: {self._calculate_total_length(mst):.2f} m")
        
        # Generate additional trees using different methods
        attempts = 0
        max_attempts = n_trees * 100
        
        while len(all_trees) < n_trees and attempts < max_attempts:
            attempts += 1
            
            # Method 1: Random spanning tree using DFS
            if attempts % 3 == 0:
                tree = self._random_spanning_tree_dfs()
            # Method 2: Modified Kruskal with randomization
            elif attempts % 3 == 1:
                tree = self._random_spanning_tree_kruskal()
            # Method 3: Random walk based
            else:
                tree = self._random_spanning_tree_walk()
            
            if tree and nx.is_connected(tree) and tree.number_of_nodes() == self.base_graph.number_of_nodes():
                tree_signature = self._get_tree_signature(tree)
                if tree_signature not in seen_trees:
                    length = self._calculate_total_length(tree)
                    all_trees.append((length, tree))
                    seen_trees.add(tree_signature)
                    if len(all_trees) % 5 == 0:
                        print(f"Generated {len(all_trees)} trees...")
        
        # Sort by length and return top n_trees
        all_trees.sort(key=lambda x: x[0])
        self.spanning_trees = [tree for _, tree in all_trees[:n_trees]]
        
        print(f"Successfully generated {len(self.spanning_trees)} unique spanning trees")
        return self.spanning_trees
    
    def _random_spanning_tree_dfs(self) -> nx.Graph:
        """Generate random spanning tree using randomized DFS"""
        tree = nx.Graph()
        tree.add_nodes_from(self.base_graph.nodes(data=True))
        
        visited = set()
        stack = [self.outlet_node]
        visited.add(self.outlet_node)
        
        while stack and len(visited) < self.base_graph.number_of_nodes():
            current = stack.pop()
            neighbors = list(self.base_graph.neighbors(current))
            np.random.shuffle(neighbors)
            
            for neighbor in neighbors:
                if neighbor not in visited:
                    tree.add_edge(current, neighbor, **self.base_graph[current][neighbor])
                    visited.add(neighbor)
                    stack.append(neighbor)
        
        return tree if len(visited) == self.base_graph.number_of_nodes() else None
    
    def _random_spanning_tree_kruskal(self) -> nx.Graph:
        """Generate spanning tree using randomized Kruskal's algorithm"""
        edges = list(self.base_graph.edges(data=True))
        # Randomize edge weights slightly
        weighted_edges = [(u, v, data['length'] * (0.8 + 0.4 * np.random.random()), data) 
                         for u, v, data in edges]
        weighted_edges.sort(key=lambda x: x[2])
        
        tree = nx.Graph()
        tree.add_nodes_from(self.base_graph.nodes(data=True))
        
        for u, v, _, data in weighted_edges:
            tree.add_edge(u, v, **data)
            if not nx.is_tree(tree):
                tree.remove_edge(u, v)
            if tree.number_of_edges() == self.base_graph.number_of_nodes() - 1:
                break
        
        return tree if nx.is_connected(tree) else None
    
    def _random_spanning_tree_walk(self) -> nx.Graph:
        """Generate spanning tree using random walk"""
        tree = nx.Graph()
        tree.add_nodes_from(self.base_graph.nodes(data=True))
        
        visited = set()
        start_node = np.random.choice(list(self.base_graph.nodes()))
        visited.add(start_node)
        
        while len(visited) < self.base_graph.number_of_nodes():
            # Pick a random visited node
            current = np.random.choice(list(visited))
            # Get unvisited neighbors
            neighbors = [n for n in self.base_graph.neighbors(current) if n not in visited]
            
            if neighbors:
                next_node = np.random.choice(neighbors)
                tree.add_edge(current, next_node, **self.base_graph[current][next_node])
                visited.add(next_node)
            
        return tree if nx.is_connected(tree) else None
    
    def _get_tree_signature(self, tree: nx.Graph) -> frozenset:
        """Get unique signature for a tree based on its edges"""
        return frozenset(frozenset([u, v]) for u, v in tree.edges())
    
    def _calculate_total_length(self, graph: nx.Graph) -> float:
        """Calculate total length of all edges"""
        return sum(data['length'] for _, _, data in graph.edges(data=True))

class SewerHydraulics:
    
    """Calculate sewer hydraulic parameters"""
    
    def __init__(self, manning_n: float = 0.013):
        self.n = manning_n
        
    def calculate_flow_parameters(self, Q: float, D: float, S: float) -> Dict:
        """
        Calculate hydraulic parameters for partially full circular sewer
        Q: discharge (m3/s)
        D: diameter (m)
        S: slope (dimensionless)
        
        Returns None if parameters are invalid or calculation fails
        """
        try:
            # Input validation
            if Q is None or D is None or S is None:
                return None
            
            # Check for zero or negative values
            if Q <= 0:
                return None  # Zero or negative discharge
            
            if D <= 0:
                return None  # Invalid diameter
            
            if S <= 0:
                return None  # Invalid slope
            
            if self.n <= 0:
                return None  # Invalid Manning's n
            
            # Equation 3: K = Q*n*D^(-8/3)*S^(-1/2)
            try:
                K = Q * self.n * (D ** (-8/3)) * (S ** (-0.5))
            except (ZeroDivisionError, ValueError, OverflowError):
                return None
            
            # Check for invalid K values
            if not np.isfinite(K) or K <= 0:
                return None
            
            # Ensure K < 1/π (constraint from equation 4)
            if K >= 1/np.pi:
                return None
            
            # Equation 4: Calculate θ
            try:
                inner_sqrt = np.sqrt(np.pi * K)
                if inner_sqrt > 1:  # Check domain for next sqrt
                    return None
                
                middle_sqrt = np.sqrt(1 - inner_sqrt)
                if middle_sqrt > 1:  # Check domain for final sqrt
                    return None
                
                outer_sqrt = np.sqrt(1 - middle_sqrt)
                theta = (3 * np.pi / 2) * outer_sqrt
                
            except (ValueError, RuntimeWarning):
                return None
            
            # Validate theta
            if not np.isfinite(theta) or theta <= 0 or theta > 2 * np.pi:
                return None
            
            # Equation 5: d/D ratio
            try:
                d_D = 0.5 * (1 - np.cos(theta / 2))
            except (ValueError, OverflowError):
                return None
            
            # Validate d/D ratio
            if not np.isfinite(d_D) or d_D < 0 or d_D > 1:
                return None
            
            # Equation 6: hydraulic radius
            try:
                sin_theta = np.sin(theta)
                if not np.isfinite(sin_theta):
                    return None
                
                r = (D / 4) * ((theta - sin_theta) / theta)
            except (ZeroDivisionError, ValueError, OverflowError):
                return None
            
            # Validate hydraulic radius
            if not np.isfinite(r) or r <= 0 or r > D:
                return None
            
            # Calculate velocity: V = (1/n) * R^(2/3) * S^(1/2)
            try:
                V = (1 / self.n) * (r ** (2/3)) * (S ** 0.5)
            except (ZeroDivisionError, ValueError, OverflowError):
                return None
            
            # Validate velocity
            if not np.isfinite(V) or V <= 0:
                return None
            
            # Calculate flow depth
            d = d_D * D
            if not np.isfinite(d) or d < 0 or d > D:
                return None
            
            # All calculations successful, return results
            return {
                'K': K,
                'theta': theta,
                'd_D': d_D,
                'r': r,
                'V': V,
                'd': d
            }
            
        except Exception as e:
            # Catch any unexpected errors
            # print(f"Warning: Hydraulic calculation failed: {str(e)}")
            return None

class CostCalculator:
    """Calculate costs for sewer network components"""
    
    def __init__(self):
        # Extended cost data with larger diameters
        self.pipe_costs = {
            0.2: 518, 0.25: 724, 0.3: 973, 0.35: 1600,
            0.4: 1850, 0.45: 2150, 0.5: 2520, 0.6: 2600, 
            0.7: 2900, 0.8: 3500, 0.9: 4000, 1.0: 5000, 
            1.5: 10000
        }
        
        self.manhole_costs = {
            (0, 1): 11800, (1, 2): 23100, (2, 3): 40000,
            (3, 4): 54600, (4, 5): 69200, (5, 6): 77500
        }
        
        self.earthwork_costs = {
            (0, 1.5): 203, (1.5, 3.0): 233.5,
            (3.0, 4.5): 299, (4.5, 6.0): 405
        }
        
    def get_pipe_cost(self, diameter: float, length: float) -> float:
        """Get cost per meter for pipe diameter"""
        available_diameters = sorted(self.pipe_costs.keys())
        selected_d = min(available_diameters, key=lambda x: abs(x - diameter))
        if selected_d < diameter:
            idx = available_diameters.index(selected_d)
            if idx < len(available_diameters) - 1:
                selected_d = available_diameters[idx + 1]
        return self.pipe_costs[selected_d] * length
    
    def get_manhole_cost(self, depth: float) -> float:
        """Get manhole cost based on depth"""
        for (d_min, d_max), cost in self.manhole_costs.items():
            if d_min < depth <= d_max:
                return cost
        return 77500  # Maximum cost
    
    def get_earthwork_cost(self, depth: float, volume: float) -> float:
        """Get earthwork cost"""
        for (d_min, d_max), cost in self.earthwork_costs.items():
            if d_min < depth <= d_max:
                return cost * volume
        return 405 * volume  # Maximum cost
    
    

class ModifiedPSO:
    """Modified Particle Swarm Optimization for sewer component sizing"""
    
    def __init__(self, n_particles: int, n_iterations: int, n_dimensions: int):
        self.n_particles = n_particles
        self.n_iterations = n_iterations
        self.n_dimensions = n_dimensions
        
        # PSO parameters
        self.w_max = 0.7
        self.w_min = 0.2
        self.c1_max = 2.0
        self.c1_min = 0.5
        self.c2_max = 2.0
        self.c2_min = 0.5
        
        # Initialize particles
        self.particles = None
        self.velocities = None
        self.pbest = None
        self.pbest_costs = None
        self.gbest = None
        self.gbest_cost = float('inf')
        
    def optimize(self, cost_function, bounds: List[Tuple[float, float]]) -> Tuple:
        """
        Optimize using Modified PSO
        cost_function: function to minimize
        bounds: list of (min, max) for each dimension
        """
        # Initialize particles randomly within bounds
        self.particles = np.random.uniform(
            low=[b[0] for b in bounds],
            high=[b[1] for b in bounds],
            size=(self.n_particles, self.n_dimensions)
        )
        
        # Initialize velocities
        v_max = [(b[1] - b[0]) * 0.15 for b in bounds]
        self.velocities = np.random.uniform(
            low=[-vm for vm in v_max],
            high=v_max,
            size=(self.n_particles, self.n_dimensions)
        )
        
        # Initialize personal best
        self.pbest = self.particles.copy()
        self.pbest_costs = np.array([cost_function(p) for p in self.particles])
        
        # Initialize global best
        best_idx = np.argmin(self.pbest_costs)
        self.gbest = self.pbest[best_idx].copy()
        self.gbest_cost = self.pbest_costs[best_idx]
        
        # Optimization loop
        for t in range(self.n_iterations):
            # Update inertia weight and acceleration coefficients (Eqs 20-22)
            w = self.w_max - (self.w_max - self.w_min) * t / self.n_iterations
            c1 = self.c1_max - (self.c1_max - self.c1_min) * t / self.n_iterations
            c2 = self.c2_max - (self.c2_max - self.c2_min) * t / self.n_iterations
            
            for i in range(self.n_particles):
                # Random coefficients
                r1 = np.random.random(self.n_dimensions)
                r2 = np.random.random(self.n_dimensions)
                
                # Update velocity (Eq 19)
                self.velocities[i] = (
                    w * self.velocities[i] +
                    c1 * r1 * (self.pbest[i] - self.particles[i]) +
                    c2 * r2 * (self.gbest - self.particles[i])
                )
                
                # Limit velocity
                for d in range(self.n_dimensions):
                    self.velocities[i, d] = np.clip(
                        self.velocities[i, d],
                        -v_max[d], v_max[d]
                    )
                
                # Update position (Eq 18)
                self.particles[i] = self.particles[i] + self.velocities[i]
                
                # Apply bounds
                for d in range(self.n_dimensions):
                    self.particles[i, d] = np.clip(
                        self.particles[i, d],
                        bounds[d][0], bounds[d][1]
                    )
                
                # Evaluate fitness
                cost = cost_function(self.particles[i])
                
                # Update personal best
                if cost < self.pbest_costs[i]:
                    self.pbest[i] = self.particles[i].copy()
                    self.pbest_costs[i] = cost
                    
                    # Update global best
                    if cost < self.gbest_cost:
                        self.gbest = self.particles[i].copy()
                        self.gbest_cost = cost
            
            if (t + 1) % 10 == 0:
                print(f"Iteration {t+1}/{self.n_iterations}, Best Cost: {self.gbest_cost:.2f}")
        
        return self.gbest, self.gbest_cost

    
    
class SewerNetworkOptimizer:
    """Main optimizer combining spanning tree generation and PSO"""
    
    def __init__(self, base_graph: nx.Graph, nodes: Dict[int, Node], outlet_node: int):
        self.base_graph = base_graph
        self.nodes = nodes
        self.outlet_node = outlet_node
        self.hydraulics = SewerHydraulics()
        self.cost_calc = CostCalculator()
        self.best_design_details = None  # Store detailed design info
        self.optimization_history = {}  # Store optimization results for plotting
        
    def calculate_cumulative_flow(self, tree: nx.Graph) -> float:
        """Calculate total cumulative flow (CQ) for a layout (Eq 1)"""
        # Convert to directed tree first
        directed_tree = self._get_flow_direction_tree(tree)
        
        total_cq = 0.0
        
        # For each link, calculate the flow and add to cumulative total
        for u, v in directed_tree.edges():
            # Calculate flow in this link by summing all upstream contributions
            link_flow = self._calculate_link_flow_directed(directed_tree, u, v)
            total_cq += link_flow
        
        return total_cq
    
    def _calculate_link_flow(self, tree: nx.Graph, u: int, v: int) -> float:
        """Calculate flow in a link by summing upstream contributions (simplified)"""
        # This is a fallback for undirected trees
        return abs(self.nodes[u].wastewater_contribution + 
                   self.nodes[v].wastewater_contribution) / 1000  # Convert l/s to m3/s
    
    def optimize_layout_sequence(self, n_layouts: int = 10, 
                                 pso_particles: int = 100,
                                 pso_iterations: int = 30):
        """Main optimization routine"""
        # Step 1: Generate spanning trees
        print("\n" + "=" * 80)
        print("STEP 1: Generating Spanning Trees")
        print("=" * 80)
        tree_gen = SpanningTreeGenerator(self.base_graph, self.outlet_node)
        trees = tree_gen.generate_spanning_trees(n_layouts)
        
        if len(trees) < n_layouts:
            print(f"⚠ Warning: Only generated {len(trees)} unique trees out of {n_layouts} requested")
        
        # Step 2: Calculate CQ for each layout
        print("\n" + "=" * 80)
        print("STEP 2: Calculating Cumulative Flows (CQ)")
        print("=" * 80)
        layouts_with_cq = []
        for i, tree in enumerate(trees):
            cq = self.calculate_cumulative_flow(tree)
            length = sum(data['length'] for _, _, data in tree.edges(data=True))
            layouts_with_cq.append((cq, tree, length))
            print(f"Layout {i+1}: CQ = {cq:.4f} m³/s ({cq*1000:.2f} l/s), Length = {length:.2f} m")
        
        # Step 3: Sort by CQ
        print("\n" + "=" * 80)
        print("STEP 3: Sorting Layouts by Cumulative Flow")
        print("=" * 80)
        layouts_with_cq.sort(key=lambda x: x[0])
        
        print(f"\nLayouts sorted (ascending CQ):")
        for i, (cq, _, length) in enumerate(layouts_with_cq[:10]):
            print(f"  Rank {i+1}: CQ = {cq:.4f} m³/s ({cq*1000:.2f} l/s), Length = {length:.2f} m")
        
        # Step 4: Optimize each layout with Modified PSO
        print("\n" + "=" * 80)
        print("STEP 4: Optimizing Component Sizes with Modified PSO")
        print("=" * 80)
        results = []
        
        for i, (cq, tree, length) in enumerate(layouts_with_cq):
            print(f"\n--- Optimizing Layout {i+1}/{len(layouts_with_cq)} ---")
            print(f"    CQ = {cq:.4f} m³/s, Length = {length:.2f} m")
            cost, design_details = self._optimize_component_sizing(tree, pso_particles, pso_iterations)
            results.append((i+1, cq, cost, tree, design_details))
            print(f"    Final Cost: Rs. {cost:,.2f}")
        
        # Sort results by cost (lowest cost = best design)
        print("\n" + "=" * 80)
        print("STEP 5: Sorting Layouts by Total Cost")
        print("=" * 80)
        results.sort(key=lambda x: x[2])  # Sort by cost (index 2)
        
        print("\nLayouts ranked by cost (Best = Minimum Cost):")
        for rank, (layout_num, cq, cost, _, _) in enumerate(results[:10], 1):
            print(f"  Rank {rank}: Layout {layout_num}, Cost = Rs. {cost:,.2f}, CQ = {cq:.4f} m³/s")
        
        # Store the best design details (minimum cost)
        if results:
            self.best_design_details = results[0][4]  # Details from best layout (lowest cost)
            
        return results
    
    def _optimize_component_sizing(self, tree: nx.Graph, 
                                   n_particles: int, 
                                   n_iterations: int) -> Tuple[float, Dict]:
        """Optimize component sizes for a given layout using Modified PSO"""
        n_links = tree.number_of_edges()
        
        # Each link has 2 variables: diameter index and slope
        n_dimensions = n_links * 2
        
        # Define bounds with extended diameter range
        available_diameters = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.5]
        bounds = []
        for _ in range(n_links):
            bounds.append((0, len(available_diameters) - 1))  # Diameter index
            bounds.append((0.0004, 0.02))  # Slope range
        
        # Define cost function
        def cost_function(x):
            cost, _ = self._evaluate_design(tree, x, available_diameters)
            return cost
        
        # Run PSO
        pso = ModifiedPSO(n_particles, n_iterations, n_dimensions)
        best_solution, best_cost = pso.optimize(cost_function, bounds)
        
        # Get detailed design for best solution
        _, design_details = self._evaluate_design(tree, best_solution, available_diameters)
        
        return best_cost, design_details
    
    def _get_flow_direction_tree(self, tree: nx.Graph) -> nx.DiGraph:
        """Convert undirected tree to directed tree with flow from sources to outlet"""
        # Create directed graph with flow towards outlet
        directed_tree = nx.DiGraph()
        directed_tree.add_nodes_from(tree.nodes(data=True))
        
        # BFS from outlet to determine flow direction
        visited = set()
        queue = [self.outlet_node]
        visited.add(self.outlet_node)
        parent = {self.outlet_node: None}
        
        while queue:
            current = queue.pop(0)
            for neighbor in tree.neighbors(current):
                if neighbor not in visited:
                    # Flow goes FROM neighbor TO current (towards outlet)
                    directed_tree.add_edge(neighbor, current, **tree[neighbor][current])
                    parent[neighbor] = current
                    visited.add(neighbor)
                    queue.append(neighbor)
        
        return directed_tree
    
    def _get_topological_order(self, directed_tree: nx.DiGraph) -> List[Tuple]:
        """Get edges in topological order (upstream to downstream)"""
        # Find leaf nodes (sources - no incoming edges)
        in_degree = {node: directed_tree.in_degree(node) for node in directed_tree.nodes()}
        
        # Start with nodes that have no predecessors (leaf nodes)
        queue = [node for node, degree in in_degree.items() if degree == 0]
        ordered_edges = []
        
        while queue:
            current = queue.pop(0)
            # Get all outgoing edges (downstream direction)
            for successor in directed_tree.successors(current):
                edge_data = directed_tree[current][successor]
                ordered_edges.append((current, successor, edge_data))
                
                # Decrease in-degree and add to queue if all predecessors processed
                in_degree[successor] -= 1
                if in_degree[successor] == 0:
                    queue.append(successor)
        
        return ordered_edges
    
    def _evaluate_design(self, tree: nx.Graph, design: np.ndarray, 
                        available_diameters: List[float]) -> Tuple[float, Dict]:
        """Evaluate total cost and constraints for a design"""
        total_cost = 0.0
        penalty = 0.0
        design_details = []
        
        # Convert to directed tree for proper flow direction
        directed_tree = self._get_flow_direction_tree(tree)
        
        # Get edges in topological order (upstream to downstream)
        ordered_edges = self._get_topological_order(directed_tree)
        
        # Map to store diameter at each node for progressive diameter constraint
        node_diameter = {}
        
        # Create edge index mapping
        edge_to_index = {}
        for i, (u, v, _) in enumerate(ordered_edges):
            edge_to_index[(u, v)] = i
        
        for i, (u, v, data) in enumerate(ordered_edges):
            # Extract design variables
            d_idx = int(round(design[i * 2]))
            d_idx = np.clip(d_idx, 0, len(available_diameters) - 1)
            diameter = available_diameters[d_idx]
            slope = design[i * 2 + 1]
            
            # Progressive diameter constraint (Eq. 11): D_current >= D_preceding
            # For each upstream node feeding into current link, check its diameter
            max_preceding_diameter = 0.0
            upstream_nodes_with_diameter = []
            
            for pred in directed_tree.predecessors(u):
                if pred in node_diameter:
                    upstream_nodes_with_diameter.append((pred, node_diameter[pred]))
                    max_preceding_diameter = max(max_preceding_diameter, node_diameter[pred])
            
            # Enforce progressive diameter constraint
            if max_preceding_diameter > 0:
                if diameter < max_preceding_diameter:
                    # Must use at least the maximum preceding diameter
                    diameter = max_preceding_diameter
                    # Find closest available diameter that satisfies constraint
                    valid_diameters = [d for d in available_diameters if d >= max_preceding_diameter]
                    if valid_diameters:
                        diameter = min(valid_diameters)
            
            # Store diameter at downstream node for this link
            node_diameter[v] = diameter
            
            length = data['length']
            flow = self._calculate_link_flow_directed(directed_tree, u, v)
            
            # Calculate hydraulics
            params = self.hydraulics.calculate_flow_parameters(flow, diameter, slope)
            
            if params is None:
                penalty += 1e8
                # Store design even if invalid
                design_details.append({
                    'link': i + 1,
                    'from_node': u,
                    'to_node': v,
                    'length': length,
                    'diameter': diameter,
                    'slope': slope,
                    'slope_ratio': f"1 in {int(1/slope) if slope > 0 else 'inf'}",
                    'flow': flow,
                    'velocity': None,
                    'd_D': None,
                    'd': None,
                    'status': 'Invalid - K >= 1/π'
                })
                continue
            
            # Calculate costs
            pipe_cost = self.cost_calc.get_pipe_cost(diameter, length)
            
            # Simplified depth calculation
            avg_depth = 1.5  # Simplified
            manhole_cost = self.cost_calc.get_manhole_cost(avg_depth)
            earthwork_cost = self.cost_calc.get_earthwork_cost(avg_depth, length * 1.0)
            
            link_cost = pipe_cost + manhole_cost + earthwork_cost
            total_cost += link_cost
            
            # Check constraints
            V = params['V']
            d_D = params['d_D']
            d = params['d']
            
            # Track constraint violations
            violations = []
            
            # Check progressive diameter constraint
            if max_preceding_diameter > 0:
                original_d_idx = int(round(design[i * 2]))
                original_d_idx = np.clip(original_d_idx, 0, len(available_diameters) - 1)
                original_diameter = available_diameters[original_d_idx]
                
                if original_diameter < max_preceding_diameter - 0.001:
                    penalty += 1e6 * (max_preceding_diameter - original_diameter)
                    violations.append(f"D_prog: needed {diameter*1000:.0f}mm")
            
            # Velocity constraints
            if V < 0.6:
                if flow >= 0.0014:  # As per paper
                    penalty += 1e8 * (0.6 - V)
                    violations.append(f"V < 0.6 m/s")
            if V > 3.0:
                penalty += 1e8 * (V - 3.0)
                violations.append(f"V > 3.0 m/s")
            
            # Depth ratio constraint
            if d_D > 0.8:
                penalty += 1e8 * (d_D - 0.8)
                violations.append(f"d/D > 0.8")
            
            # Cover depth (simplified check)
            if avg_depth < 0.9:
                penalty += 1e8 * (0.9 - avg_depth)
                violations.append(f"Cover < 0.9 m")
            if avg_depth > 5.0:
                penalty += 1e8 * (avg_depth - 5.0)
                violations.append(f"Cover > 5.0 m")
            
            # Store design details
            design_details.append({
                'link': i + 1,
                'from_node': u,
                'to_node': v,
                'length': length,
                'diameter': diameter,
                'slope': slope,
                'slope_ratio': f"1 in {int(1/slope) if slope > 0 else 'inf'}",
                'flow': flow,
                'flow_lps': flow * 1000,  # Convert to l/s
                'velocity': V,
                'd_D': d_D,
                'd': d,
                'link_cost': link_cost,
                'status': 'OK' if not violations else ', '.join(violations),
                'max_preceding_diameter': max_preceding_diameter if max_preceding_diameter > 0 else None
            })
        
        return total_cost + penalty, design_details
    
    def _calculate_link_flow_directed(self, directed_tree: nx.DiGraph, u: int, v: int) -> float:
        """Calculate flow in a directed link by summing all upstream contributions"""
        # Get all nodes upstream of u (including u itself)
        upstream_nodes = set()
        
        def find_upstream(node):
            upstream_nodes.add(node)
            # Find all predecessors in directed tree
            for pred in directed_tree.predecessors(node):
                find_upstream(pred)
        
        find_upstream(u)
        
        # Sum all wastewater contributions from upstream nodes
        total_flow = sum(self.nodes[node].wastewater_contribution 
                        for node in upstream_nodes 
                        if self.nodes[node].wastewater_contribution > 0)
        
        return total_flow / 1000  # Convert l/s to m3/s

    def run_sensitivity_analysis(self, best_tree: nx.Graph, 
                                 swarm_sizes: List[int] = [200, 400, 600, 800, 1000],
                                 iterations_list: List[int] = [30, 60, 90, 120]):
        """
        Run sensitivity analysis for different swarm sizes and iterations
        Returns results for plotting
        """
        print("\n" + "=" * 80)
        print("SENSITIVITY ANALYSIS: Swarm Size vs Cost")
        print("=" * 80)
        
        results = {}
        total_runs = len(swarm_sizes) * len(iterations_list)
        current_run = 0
        
        for iterations in iterations_list:
            results[iterations] = {}
            for swarm_size in swarm_sizes:
                current_run += 1
                print(f"\n[{current_run}/{total_runs}] Testing: Swarm={swarm_size}, Iterations={iterations}")
                
                # Run optimization
                cost, _ = self._optimize_component_sizing(best_tree, swarm_size, iterations)
                results[iterations][swarm_size] = cost
                
                print(f"    Result: Rs. {cost:,.2f}")
                
                
        
        # Store in history
        self.optimization_history = results
        
        return results
    
    
    def plot_sensitivity_analysis(self, results: Dict = None, save_path: str = 'output'):
        """
        Plot swarm size vs cost for different iterations
        """
        if results is None:
            results = self.optimization_history
        
        if not results:
            print("No results to plot. Run sensitivity analysis first.")
            return
        
        # Create figure with larger size
        plt.figure(figsize=(12, 8))
        
        # Define colors and markers for different iterations
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
        markers = ['o', 's', '^', 'D']
        
        # Extract data and plot
        iterations_list = sorted(results.keys())
        
        for idx, iterations in enumerate(iterations_list):
            swarm_sizes = sorted(results[iterations].keys())
            costs = [results[iterations][size] for size in swarm_sizes]
            
            # Convert costs to millions for better readability
            costs_millions = [c / 1e6 for c in costs]
            
            plt.plot(swarm_sizes, costs_millions, 
                    marker=markers[idx % len(markers)], 
                    color=colors[idx % len(colors)],
                    linewidth=2, 
                    markersize=8,
                    label=f'{iterations} Iterations')
        
        plt.xlabel('Swarm Size', fontsize=12, fontweight='bold')
        plt.ylabel('Total Cost (Million Rs.)', fontsize=12, fontweight='bold')
        plt.title('Sensitivity Analysis: Swarm Size vs Total Cost\nat Different Iterations', 
                 fontsize=14, fontweight='bold')
        plt.legend(fontsize=10, loc='best')
        plt.grid(True, alpha=0.3, linestyle='--')
        plt.xticks(swarm_sizes if swarm_sizes else [200, 400, 600, 800, 1000])
        
        # Add minor gridlines
        plt.minorticks_on()
        plt.grid(which='minor', alpha=0.2, linestyle=':')
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"\nPlot saved to: {save_path}")
        
        plt.show()
        
        # Print summary table
        print("\n" + "=" * 80)
        print("SENSITIVITY ANALYSIS RESULTS TABLE")
        print("=" * 80)
        print(f"\n{'Swarm Size':<12}", end='')
        for iterations in iterations_list:
            print(f"{'Iter=' + str(iterations):<20}", end='')
        print()
        print("-" * 80)
        
        if iterations_list and results[iterations_list[0]]:
            swarm_sizes = sorted(results[iterations_list[0]].keys())
            for swarm_size in swarm_sizes:
                print(f"{swarm_size:<12}", end='')
                for iterations in iterations_list:
                    cost = results[iterations][swarm_size]
                    print(f"Rs. {cost:>15,.2f}   ", end='')
                print()
        
        # Find best configuration
        print("\n" + "=" * 80)
        print("BEST CONFIGURATIONS")
        print("=" * 80)
        
        min_cost = float('inf')
        best_config = None
        
        for iterations in iterations_list:
            for swarm_size, cost in results[iterations].items():
                if cost < min_cost:
                    min_cost = cost
                    best_config = (swarm_size, iterations)
        
        if best_config:
            print(f"\n★ Overall Best: Swarm Size = {best_config[0]}, Iterations = {best_config[1]}")
            print(f"  Minimum Cost: Rs. {min_cost:,.2f}")
            
        # Best for each iteration level
        print("\n📊 Best Swarm Size for Each Iteration Level:")
        for iterations in iterations_list:
            costs = results[iterations]
            best_swarm = min(costs, key=costs.get)
            best_cost = costs[best_swarm]
            print(f"  {iterations} Iterations: Swarm = {best_swarm}, Cost = Rs. {best_cost:,.2f}")
        
        

        
        

def parse_sewer_file(filepath):
    """
    Parse the same file format as CedritosNorte.txt
    Returns:
        nodes_df: DataFrame with columns ['id','flow','x','y','elevation']
        edges_df: DataFrame with columns ['u','v']
    """
    with open(filepath, "r") as f:
        lines = [ln.strip() for ln in f if ln.strip()]

    # find "Manholes" line index
    idx = 0
    if not lines[0].lower().startswith("manholes"):
        raise ValueError("File must start with 'Manholes <count>'")
    manholes_count = int(lines[0].split()[1])
    manhole_lines = lines[1: 1 + manholes_count]

    nodes = []
    for ln in manhole_lines:
        parts = ln.split()
        if len(parts) < 5:
            raise ValueError("Expected 5 columns in manholes rows")
        nid = int(parts[0])
        flow = float(parts[1])
        x = float(parts[2])
        y = float(parts[3])
        elev = float(parts[4])
        nodes.append((nid, flow, x, y, elev))
    nodes_df = pd.DataFrame(nodes, columns=["id", "flow", "x", "y", "elevation"]).set_index("id")

    # next line after manholes block should be "Sections <count>"
    sec_line_idx = 1 + manholes_count
    if not lines[sec_line_idx].lower().startswith("sections"):
        raise ValueError("Expected 'Sections <count>' after Manholes block")
    sec_count = int(lines[sec_line_idx].split()[1])
    section_lines = lines[sec_line_idx + 1: sec_line_idx + 1 + sec_count]

    edges = []
    for ln in section_lines:
        parts = ln.split()
        if len(parts) < 2:
            continue
        u, v = int(parts[0]), int(parts[1])
        edges.append((u, v))
    edges_df = pd.DataFrame(edges, columns=["u", "v"])
    return nodes_df, edges_df



def build_weighted_graph(nodes_df, edges_df):
    """
    Build an undirected graph with Euclidean distance weights between node coords.
    Returns networkx.Graph() and a pos dict {node: (x,y)} for plotting.
    nodes_df: index = node id, columns include ['x','y','flow','elevation'] (flow/elevation optional)
    edges_df: two columns (u,v) or index can be ignored; each row is (u,v)
    """
    G = nx.Graph()
    outlet_id=None
    
    # Add nodes with attributes and build pos dict
    for nid, row in nodes_df.iterrows():

        x = float(row["x"])
        y = float(row["y"])
        # safe-get flow and elevation (set to 0 if missing)
        flow = float(row["flow"]) if "flow" in row and not pd.isna(row["flow"]) else 0.0
        #print(flow)
        
        #outlet node
        if(flow<0):
            outlet_id=nid
            
        elev = float(row["elevation"]) if "elevation" in row and not pd.isna(row["elevation"]) else None
        G.add_node(nid, x=x, y=y, ground_level=elev,flow=flow)

    # Add edges with length weight
    for _, (u, v) in edges_df.iterrows():
        if u not in nodes_df.index or v not in nodes_df.index:
            continue
        xu, yu = nodes_df.loc[u, ["x", "y"]]
        xv, yv = nodes_df.loc[v, ["x", "y"]]
        length = math.hypot(float(xu) - float(xv), float(yu) - float(yv))
        G.add_edge(u, v, length=length)
        
            
    print(f"\nTree Network Summary:")
    print(f"="*20)
    print(f"Total Manholes: {G.number_of_nodes()}")
    print(f"Total Sections: {G.number_of_edges()}")
    print(f"Outlet Node: {outlet_id}")

        

    return G,outlet_id

import os, csv

def safe_float(value, decimals=None):
    if value is None or value == "":
        return ""
    try:
        v = float(value)
        return round(v, decimals) if decimals is not None else v
    except:
        return ""

import os, csv


def safe_float(value, decimals=None):
    if value is None or value == "":
        return ""
    try:
        v = float(value)
        return round(v, decimals) if decimals is not None else v
    except:
        return ""

def save_results_with_input_details(
    Input_file: str,
    nodes_df,
    nodes_data: dict,
    best_details: list,
    output_dir: str = "output"
):
    base_name = os.path.splitext(os.path.basename(Input_file))[0]
    output_file = os.path.join(output_dir, f"{base_name}_results.csv")

    required_cols = ["x", "y", "elevation", "flow"]
    for col in required_cols:
        if col not in nodes_df.columns:
            raise KeyError(f"Column '{col}' not found in nodes_df")

    fieldnames = [
        'link', 'from_node', 'to_node', 'length', 'diameter', 'slope', 'slope_ratio',
        'flow_lps', 'velocity', 'd_D', 'd', 'status', 'max_preceding_diameter',
        'input_flow_lps', 'x', 'y', 'z'
    ]

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        for detail in best_details:
            from_node = detail.get('from_node')
            node_info = nodes_df.loc[from_node] if from_node in nodes_df.index else None

            input_flow = x = y = z = ""
            if node_info is not None:
                input_flow = safe_float(node_info['flow'] * 1000, 3)
                x = safe_float(node_info['x'])
                y = safe_float(node_info['y'])
                z = safe_float(node_info['elevation'])

            flow_lps = detail.get('flow_lps')
            if flow_lps is None and 'flow' in detail:
                flow_lps = detail['flow'] * 1000

            writer.writerow({
                'link': detail.get('link', ''),
                'from_node': from_node,
                'to_node': detail.get('to_node', ''),
                'input_flow_lps': input_flow,
                'x': x,
                'y': y,
                'z': z,
                'length': safe_float(detail.get('length'), 2),
                'diameter': detail.get('diameter', ''),
                'slope': safe_float(detail.get('slope'), 2),
                'slope_ratio': detail.get('slope_ratio', ''),
                'flow_lps': safe_float(flow_lps, 3),
                'velocity': safe_float(detail.get('velocity'), 2),
                'd_D': safe_float(detail.get('d_D'), 3),
                'd': safe_float(detail.get('d'), 3),
                'status': detail.get('status', ''),
                'max_preceding_diameter': detail.get('max_preceding_diameter', '')
            })

    print(f"✅ Results with input details saved to: {output_file}")
    return output_file



def plot_graph_with_coords(G, title="Tree Plot", show_elevation=False, show_lengths=True, 
                          show_edge_flows=False, figsize=(10, 8), save_path=None):

    plt.figure(figsize=figsize)
    ax = plt.gca()
    
    # extract positions of manholes
    pos = {}
    for n, data in G.nodes(data=True):
        if "x" in data and "y" in data:
            pos[n] = (float(data["x"]), float(data["y"]))
    
    # Node sizes: scale flow (add small base so zero-flow nodes are visible)
    flows = nx.get_node_attributes(G, "flow")
    
    # Edge widths: normalize lengths to a reasonable thickness
    lengths = nx.get_edge_attributes(G, "length")
    edge_flows = nx.get_edge_attributes(G, "flow_m3s")
    
    # Draw edges and nodes at their coordinates
    nx.draw_networkx_edges(G, pos, ax=ax, width=2)
    node_collection = nx.draw_networkx_nodes(G, pos, node_size=500, ax=ax)
    nx.draw_networkx_labels(G, pos, font_size=9, ax=ax)
    
    edge_labels = {}
    for e, d in G.edges.items():
        text = []
        if show_lengths and "length" in d:
            text.append(f"L={d['length']:.2f} m")
        if show_edge_flows and "flow_m3s" in d:
            text.append(f"Q={d['flow_m3s']:.3f} m3/s")
        if text:
            edge_labels[e] = "\n".join(text)  # stacked
            
    if edge_labels:
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, label_pos=0.5)
    
    # Optional: annotate elevation next to nodes
    if show_elevation:
        elevs = nx.get_node_attributes(G, "elevation")
        for n, (x, y) in pos.items():
            ev = elevs.get(n, None)
            if ev is not None:
                ax.text(x + 0.2, y + 0.2, f"{ev:.2f}m", fontsize=7, alpha=0.8)
    
    ax.set_aspect('equal', adjustable='datalim')   # preserve scale
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    plt.title(title)
    plt.tight_layout()
    
    # Save figure if path is provided
    save_path="output/result_layout.png"
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Figure saved to: {save_path}")
    
    plt.show()


    
def _split_cols(line: str):
    """Split a line by whitespace/tabs but preserve negative signs and decimals."""
    return re.split(r'\s+', line.strip())

def parse_sewer_file_1(filepath: str):
    """
    Parse a sewer network .txt file in the 'CedritosNorte' or 'Li' format.

    Returns:
        nodes_df: DataFrame indexed by 'id' with columns ['flow','x','y','elevation']
        edges_df: DataFrame with columns ['u','v']
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = [ln.strip() for ln in f if ln.strip()]

    # ---- MANHOLES BLOCK ----
    idx = 0
    while idx < len(lines) and not lines[idx].lower().startswith("manholes"):
        idx += 1
    if idx >= len(lines):
        raise ValueError("File must start with 'Manholes <count>'")
    manholes_count = int(_split_cols(lines[idx])[1])

    # detect header row (like: ID X Y Z INFLOW)
    manhole_start = idx + 1
    if re.search(r'[A-Za-z]', lines[manhole_start]):
        manhole_lines = lines[manhole_start + 1: manhole_start + 1 + manholes_count]
    else:
        manhole_lines = lines[manhole_start: manhole_start + manholes_count]

    nodes = []
    for ln in manhole_lines:
        parts = _split_cols(ln)
        if len(parts) < 5:
            continue
        nid = int(parts[0])
        x = float(parts[1])
        y = float(parts[2])
        elev = float(parts[3])
        flow = float(parts[4])
        nodes.append((nid, flow, x, y, elev))

    nodes_df = pd.DataFrame(nodes, columns=["id", "flow", "x", "y", "elevation"]).set_index("id")

    # ---- SECTIONS BLOCK ----
    sec_line_idx = manhole_start + 1 + manholes_count
    while sec_line_idx < len(lines) and not lines[sec_line_idx].lower().startswith("sections"):
        sec_line_idx += 1
    if sec_line_idx >= len(lines):
        raise ValueError("Could not find 'Sections' block")

    sec_count = int(_split_cols(lines[sec_line_idx])[1])

    # detect section header like "v1 v2 slope intercept"
    section_start = sec_line_idx + 1
    if re.search(r'[A-Za-z]', lines[section_start]):
        section_lines = lines[section_start + 1: section_start + 1 + sec_count]
    else:
        section_lines = lines[section_start: section_start + sec_count]

    edges = []
    for ln in section_lines:
        parts = _split_cols(ln)
        if len(parts) < 2:
            continue
        u, v = int(parts[0]), int(parts[1])
        edges.append((u, v))

    edges_df = pd.DataFrame(edges, columns=["u", "v"])

    return nodes_df, edges_df

def get_pso_settings():
    while True:
        default = input("Use default settings (Y/N): ").strip().upper()

        if default == "N":
            try:
                n_layouts = int(input("Enter number of top layouts: "))
                pso_particles = int(input("Enter number of PSO particles: "))
                pso_iterations = int(input("Enter Max Iterations: "))
                break
            except ValueError:
                print("⚠️ Invalid input! Please enter numeric values.\n")
        
        elif default == "Y":
            n_layouts = 8
            pso_particles = 10
            pso_iterations = 10
            break
        
        else:
            print("⚠️ Please enter only Y or N.\n")

    print("\n✅ Settings Applied Successfully:")
    print(f"Top Layouts     : {n_layouts}")
    print(f"PSO Particles   : {pso_particles}")
    print(f"Max Iterations  : {pso_iterations}")
    
    return n_layouts, pso_particles, pso_iterations

def ask_and_run_sensitivity(best_tree):
    run_choice = input("Do you want to run Sensitivity Analysis? (Y/N): ").strip().upper()

    if run_choice == "N":
        print("\n⏭️ Sensitivity analysis skipped.")
        return None

    elif run_choice == "Y":
        # Ask for default/custom settings
        default = input("Use default sensitivity settings? (Y/N): ").strip().upper()

        if default == "N":
            try:
                swarm_sizes = list(map(int, input("Enter swarm sizes (comma-separated): ").split(",")))
                iterations_list = list(map(int, input("Enter iteration list (comma-separated): ").split(",")))
            except ValueError:
                print("⚠️ Invalid input! Using default values instead.")
                swarm_sizes = [200, 400, 600, 800, 1000]
                iterations_list = [30, 60, 90, 120]
        else:
            swarm_sizes = [200, 400, 600, 800, 1000]
            iterations_list = [30, 60, 90, 120]

        print("\n⚙️ Running Sensitivity Analysis with settings:")
        print(f"Swarm Sizes      : {swarm_sizes}")
        print(f"Iteration Counts : {iterations_list}\n")

        sensitivity_results = optimizer.run_sensitivity_analysis(
            best_tree,
            swarm_sizes=swarm_sizes,
            iterations_list=iterations_list
        )
        
        
        
        # Plot results
        print("\nGenerating sensitivity analysis plot...")
        optimizer.plot_sensitivity_analysis(sensitivity_results, save_path='output/sensitivity_analysis.png')
        print("DETAILED OPTIMAL DESIGN - BEST LAYOUT (Minimum Cost)")
        print("=" * 120)

        print("✅ Sensitivity Analysis Completed Successfully!")
        return 0

    else:
        print("⚠️ Please enter only Y or N.\n")
        return ask_and_run_sensitivity( best_tree)  # re-ask if invalid


In [ ]:
if __name__ == "__main__":
 
    #=====================Input File and parameter selection========================================
    
    Input_file='input/LiMathew_Revised.txt'
    
    Name=str(Input_file)
    
    nodes_df,edges_df=parse_sewer_file_1(Input_file) # Parse the input file
    
    n_layouts, pso_particles, pso_iterations= get_pso_settings() # get the input parameters
    

    #======================================Start of optimisation======================================
    
    #Graph Building
    G,outlet_id=build_weighted_graph(nodes_df,edges_df)
    
    nodes_data = {}
    for nid, row in nodes_df.iterrows():
        nodes_data[nid]=Node(id=nid,
        wastewater_contribution=round(float(row['flow']*1000),3),
        ground_level=float(row["elevation"])
                            )
        
    print("=" * 80)
    print(f"SEWER NETWORK OPTIMIZATION - {len(nodes_df)} MANHOLE NETWORK")
    print("=" * 80)
                             
    #Flow calculations 
    total_inflow = sum(n.wastewater_contribution for n in nodes_data.values() if n.wastewater_contribution > 0)
    outlet_flow = abs(nodes_data[outlet_id].wastewater_contribution)
    print(f"  Total Inflow: {total_inflow:.2f} l/s ({total_inflow/1000:.4f} m³/s)")
    print(f"  Outlet Flow: {outlet_flow:.2f} l/s ({outlet_flow/1000:.4f} m³/s)")
    
    # Check flow balance
    print(f"  Flow Balance: {'✓ OK' if abs(total_inflow - outlet_flow) < 1 else '✗ Imbalanced'}")
    
    # Create optimizer
    optimizer = SewerNetworkOptimizer(G, nodes_data, outlet_id)
    
    # Run optimization with reasonable parameters for this size network
    print("\n" + "=" * 80)
    print(f"Starting Optimization Process for {Name}...")
    print("=" * 80)
             
        
        
        
   #===================================Algorithm============================================

    results = optimizer.optimize_layout_sequence(
        n_layouts, 
        pso_particles,
        pso_iterations
    )
    
    #========================================================================================

    
    
    print("\n" + "=" * 80)
    print("OPTIMIZATION RESULTS - ALL LAYOUTS")
    print("=" * 80)
    print(f"\n{'Rank':<6} {'Layout':<8} {'CQ (m³/s)':<12} {'CQ (l/s)':<12} {'Total Cost (Rs.)':<20}")
    print("-" * 65)
    for rank, (layout_num, cq, cost, _, _) in enumerate(results[:10], 1):
        marker = "★ BEST" if rank == 1 else ""
        print(f"{rank:<6} {layout_num:<8} {cq:<12.4f} {cq*1000:<12.2f} {cost:>18,.2f}  {marker}")

    # Display detailed design for best layout (minimum cost)
    if results:
        print("\n" + "=" * 120)

        print("\n" + "=" * 80)
        print("SENSITIVITY ANALYSIS OPTION")
        print("=" * 80)
        print("(This will test different swarm sizes and iterations)")
        print("Note: This may take significant time depending on network size")

        # For demonstration, we'll run a quick sensitivity analysis on best layout
        print("\nRunning sensitivity analysis on best layout...")

        # Get best layout tree
        best_tree = results[0][3]

        #Run sensitivity analysis with configurable parameters
#         sensitivity_results = optimizer.run_sensitivity_analysis(
#             best_tree,
#             swarm_sizes=[200, 400, 600, 800, 1000],
#             iterations_list=[30, 60, 90, 120]
#         )

#         # Plot results
#         print("\nGenerating sensitivity analysis plot...")
#         optimizer.plot_sensitivity_analysis(sensitivity_results, save_path='sensitivity_analysis.png')
#         print("DETAILED OPTIMAL DESIGN - BEST LAYOUT (Minimum Cost)")
#         print("=" * 120)

        best_layout_num, best_cq, best_cost, _, best_details = results[0]
    
    
    
        #save csv file
        
        save_results_with_input_details(Input_file, nodes_df, nodes_data, best_details)

        print(f"\n✓ Best Layout: #{best_layout_num}")
        print(f"✓ Total Cumulative Flow (CQ): {best_cq:.4f} m³/s ({best_cq*1000:.2f} l/s)")
        print(f"✓ TOTAL NETWORK COST: Rs. {best_cost:,.2f} ★ MINIMUM ★")
        print(f"✓ Number of Sewer Sections: {len(best_details)}")

        # Print header
        print(f"\n{'Pipe':<6} {'From':<6} {'To':<6} {'Length':<9} {'Diameter':<10} {'Slope':<14} "
              f"{'Flow':<11} {'Velocity':<10} {'d/D':<8} {'d (m)':<8} {'Prev D':<10} {'Status':<25}")
        print(f"{'No.':<6} {'Node':<6} {'Node':<6} {'(m)':<9} {'(mm)':<10} {'(1 in X)':<14} "
              f"{'(l/s)':<11} {'(m/s)':<10} {'':<8} {'':<8} {'(mm)':<10} {'':<25}")
        print("-" * 140)

        # Print each link
        total_length = 0
        valid_links = 0
        progressive_violations = 0

        for detail in best_details:
            link = detail['link']
            from_node = detail['from_node']
            to_node = detail['to_node']
            length = detail['length']
            diameter = detail['diameter']
            slope = detail['slope']
            slope_ratio = detail['slope_ratio']
            flow_lps = detail.get('flow_lps', detail['flow'] * 1000)
            velocity = detail['velocity']
            d_D = detail['d_D']
            d = detail['d']
            status = detail['status']
            max_preceding_diameter = detail.get('max_preceding_diameter', None)

            total_length += length
            if status == 'OK':
                valid_links += 1

            if 'D_prog' in status:
                progressive_violations += 1

            # Format values
            diameter_mm = diameter * 1000
            vel_str = f"{velocity:.3f}" if velocity is not None else "N/A"
            d_D_str = f"{d_D:.4f}" if d_D is not None else "N/A"
            d_str = f"{d:.4f}" if d is not None else "N/A"
            prev_d_str = f"{max_preceding_diameter*1000:.0f}" if max_preceding_diameter else "-"

            # Color code status
            status_display = status if len(status) <= 25 else status[:22] + "..."

            print(f"{link:<6} {from_node:<6} {to_node:<6} {length:<9.2f} {diameter_mm:<10.0f} "
                  f"{slope_ratio:<14} {flow_lps:<11.3f} {vel_str:<10} {d_D_str:<8} "
                  f"{d_str:<8} {prev_d_str:<10} {status_display:<25}")
            
        # plot the graph 
        plot_graph_with_coords(best_tree,"output_graph",show_lengths=False,show_elevation=False)

        print("-" * 140)
        print(f"Total Network Length: {total_length:.2f} m")
        print(f"Valid Links (All constraints satisfied): {valid_links}/{len(best_details)}")
        if progressive_violations > 0:
            print(f"⚠ Progressive Diameter Violations: {progressive_violations}")

        # Detailed statistics
        print("\n" + "=" * 120)
        print("DESIGN STATISTICS AND CONSTRAINT VERIFICATION")
        print("=" * 120)

        velocities = [d['velocity'] for d in best_details if d['velocity'] is not None]
        d_D_ratios = [d['d_D'] for d in best_details if d['d_D'] is not None]
        diameters = [d['diameter'] * 1000 for d in best_details]
        flows = [d.get('flow_lps', d['flow'] * 1000) for d in best_details]

        print("\n📊 HYDRAULIC PARAMETERS:")
        if velocities:
            print(f"  Velocity Statistics:")
            print(f"    • Minimum: {min(velocities):.3f} m/s")
            print(f"    • Maximum: {max(velocities):.3f} m/s")
            print(f"    • Average: {np.mean(velocities):.3f} m/s")
            print(f"    • Std Dev: {np.std(velocities):.3f} m/s")

            below_min = sum(1 for v in velocities if v < 0.6)
            above_max = sum(1 for v in velocities if v > 3.0)
            print(f"    • Links below 0.6 m/s: {below_min}")
            print(f"    • Links above 3.0 m/s: {above_max}")

        if d_D_ratios:
            print(f"\n  d/D Ratio Statistics:")
            print(f"    • Minimum: {min(d_D_ratios):.4f}")
            print(f"    • Maximum: {max(d_D_ratios):.4f}")
            print(f"    • Average: {np.mean(d_D_ratios):.4f}")
            above_08 = sum(1 for r in d_D_ratios if r > 0.8)
            print(f"    • Links above 0.8: {above_08}")

        print(f"\n  Diameter Distribution:")
        unique_diameters = sorted(set(diameters))
        for d in unique_diameters:
            count = sum(1 for dia in diameters if dia == d)
            print(f"    • {d:.0f} mm: {count} links")

        print(f"\n  Flow Statistics:")
        print(f"    • Minimum: {min(flows):.3f} l/s")
        print(f"    • Maximum: {max(flows):.3f} l/s")
        print(f"    • Average: {np.mean(flows):.3f} l/s")

        print("\n✓ CONSTRAINT LIMITS:")
        print("    • Velocity: 0.6 - 3.0 m/s")
        print("    • d/D ratio: ≤ 0.8")
        print("    • Min diameter: 200 mm")
        print("    • Cover depth: 0.9 - 5.0 m")
        print("    • Progressive diameter: D_current ≥ D_preceding (Eq. 11) - ENFORCED BY SELECTION")

        # Check if any diameters were increased to satisfy progressive constraint
        enforced_increases = [d for d in best_details if d.get('enforced_increase', False)]
        if enforced_increases:
            print(f"\n⚙ Progressive Diameter Adjustments:")
            print(f"    • {len(enforced_increases)} pipe(s) had diameter increased to satisfy Eq. 11")
            for detail in enforced_increases[:5]:  # Show first 5
                link = detail['link']
                proposed = detail.get('proposed_diameter', 0) * 1000
                actual = detail['diameter'] * 1000
                print(f"      Link {link}: {proposed:.0f}mm → {actual:.0f}mm")

        # Cost breakdown
        print("\n💰 COST SUMMARY:")
        print(f"    • Total Network Cost: Rs. {best_cost:,.2f}")
        print(f"    • Cost per meter: Rs. {best_cost/total_length:,.2f}/m")

        print("\n" + "=" * 120)
        
        
        
        #============================================RUN SESITIVITY ANALYSIS=============================================
        
        sensitivity_results = ask_and_run_sensitivity(best_tree)
